<a href="https://colab.research.google.com/github/Steve-Falkovsky/Hypencoder-Entity-Linking/blob/Professional-Structure/notebooks/fine_tune_Hypencoder_on_BC5CDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import importlib.util

# for syncing changes to notebook when changing something in VScode
# deprecated/need to update to new IPython which breaks colab
# %load_ext autoreload
# %autoreload 2



REPO_NAME = "Hypencoder-Entity-Linking"
GIT_URL = f"https://github.com/Steve-Falkovsky/{REPO_NAME}.git"
BRANCH_NAME = "Professional-Structure"



# --- COLAB SETUP ---
is_colab = importlib.util.find_spec("google.colab") is not None
if is_colab:
    print("☁️ Running in Colab...")
    if not os.path.exists(REPO_NAME):
        !git clone -b {BRANCH_NAME} --single-branch {GIT_URL}

    # Move into the downloaded repo (The Root)
    os.chdir(REPO_NAME)



# --- LOCAL SETUP ---
else:
    print("💻 Running Locally...")
    if os.path.basename(os.getcwd()) == "notebooks":
        os.chdir("..")


%pip install -q -e "./hypencoder-paper"

print(f"📍 Working Directory is now: {os.getcwd()}")
print("✅ Environment Ready!")

☁️ Running in Colab...
Cloning into 'Hypencoder-Entity-Linking'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 225 (delta 84), reused 188 (delta 52), pack-reused 0 (from 0)
Receiving objects: 100% (225/225), 647.85 KiB | 25.91 MiB/s, done.
Resolving deltas: 100% (84/84), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 9.9 MB/s eta 0:00:00
📍 Working Directory is now: /content/Hypencoder-Entity-Linking
✅ Environment Ready!


In [2]:
os.chdir("./hypencoder-paper") # make it more convinient to run the scripts

In [3]:
# loading the data
from datasets import load_dataset

dataset = load_dataset("Stevenf232/hypencoder_contrastiveLoss_nameOnly")
train_data = dataset['train']
val_data = dataset['validation']

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

(…)c5cdr_train_hypencoder_contrastive.jsonl: 0.00B [00:00, ?B/s]

bc5cdr_val_hypencoder_contrastive.jsonl: 0.00B [00:00, ?B/s]

bc5cdr_test_hypencoder_contrastive.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2654 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2559 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2656 [00:00<?, ? examples/s]

In [4]:
# saving the data to a file
train_data.to_json('data/train.jsonl', lines=True)
val_data.to_json('data/val.jsonl', lines=True)

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

307335

In [5]:
# tokenizing the data before training

# training
!python hypencoder_cb/utils/tokenizer_utils.py \
--standard_format_jsonl='data/train.jsonl' \
--output_file='data/train_tokenized.jsonl' \
--tokenizer="google-bert/bert-base-uncased" \
--add_special_tokens=True \
--query_max_length=32 \
--item_max_length=512

# validation
!python hypencoder_cb/utils/tokenizer_utils.py \
--standard_format_jsonl='data/val.jsonl' \
--output_file='data/val_tokenized.jsonl' \
--tokenizer="google-bert/bert-base-uncased" \
--add_special_tokens=True \
--query_max_length=32 \
--item_max_length=512

tokenizer_config.json: 100% 48.0/48.0 [00:00<00:00, 297kB/s]
config.json: 100% 570/570 [00:00<00:00, 4.60MB/s]
vocab.txt: 232kB [00:00, 73.8MB/s]
tokenizer.json: 466kB [00:00, 115MB/s]
2654it [00:00, 392084.91it/s]
2it [00:00, 67.07it/s]
2it [00:00, 69.56it/s]
2654it [00:00, 60740.25it/s]
2559it [00:00, 258975.12it/s]
2it [00:00, 11.05it/s]
2it [00:00, 13.21it/s]
2559it [00:00, 33762.79it/s]


Everything in the output above is [00:00] which seems quite suspicious! (or it could just be really fast)



---



# Training the hypencoder

In [6]:
!python hypencoder_cb/train/train.py hypencoder_cb/train/configs/hypencoder.2_layer_SapBERT.yaml

Traceback (most recent call last):
  File "/content/Hypencoder-Entity-Linking/hypencoder-paper/hypencoder_cb/train/train.py", line 7, in <module>
    from transformers import AutoTokenizer, Trainer, TrainingArguments
  File "/usr/local/lib/python3.12/dist-packages/transformers/__init__.py", line 27, in <module>
    from . import dependency_versions_check
  File "/usr/local/lib/python3.12/dist-packages/transformers/dependency_versions_check.py", line 16, in <module>
    from .utils.versions import require_version, require_version_core
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/__init__.py", line 37, in <module>
    from .chat_template_utils import DocstringParsingException, TypeHintParsingException, get_json_schema
  File "/usr/local/lib/python3.12/dist-packages/transformers/utils/chat_template_utils.py", line 44, in <module>
    import jinja2
  File "/usr/local/lib/python3.12/dist-packages/jinja2/__init__.py", line 9, in <module>
  File "/usr/local/lib/python3.1

In [ ]:
%pip install huggingface_hub

# push the model to HuggingFace
# the model was saved in the model directory

from huggingface_hub import upload_folder

# Path where your model files (weights, config, etc.) are saved in Colab
local_folder_path = "/content/hypencoder-paper/model/hypencoder.2_layer_finetuned_BC5CDR"

In [ ]:


# Your desired repository ID on Hugging Face (e.g., "your-username/my-generic-model")
repo_id = "Stevenf232/hypencoder_BC5CDR"

# You may need to create the repository first if it doesn't exist
from huggingface_hub import create_repo
create_repo(repo_id, exist_ok=True)

upload_folder(
    folder_path=local_folder_path,
    repo_id=repo_id,
    repo_type="model", # or "dataset" or "space"
    commit_message="Upload trained model from Colab"
)